In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pydatafabric.vault_utils import get_secrets

oracle_info = get_secrets(mount_point="datafabric",path="oracle/datafabric/oracle-db")
aws_info = get_secrets(mount_point="datafabric",path="aws/credentials/datafabric")

In [4]:
from pydatafabric.ye import get_spark

spark = get_spark(extra_jars="gs://emart-datafabric-resources/jars/ojdbc8.jar")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/28 22:34:44 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/04/28 22:34:44 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/04/28 22:34:45 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/04/28 22:34:45 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [5]:
spark.conf.set("fs.s3a.fast.upload.buffer", "disk")
spark.conf.set("fs.s3a.buffer.dir", "/tmp")
spark.conf.set("fs.s3a.access.key", aws_info["aws_access_key_id"])
spark.conf.set("fs.s3a.secret.key", aws_info["aws_secret_access_key"])
spark.conf.set("spark.sql.debug.maxToStringFields", 2000)

In [6]:
df = spark.read.format("jdbc") \
    .option("url", oracle_info['jdbc_url']) \
    .option("user", oracle_info['user']) \
    .option("password", oracle_info['password']) \
    .option("driver", "oracle.jdbc.driver.OracleDriver") \
    .option("dbtable", "all_tables") \
    .load()

In [7]:
df.show()

+-----+----------------+---------------+--------------+--------+------+-------------+-------------+------------+--------------+-----------------+------------------+------------+--------------------+------------+------------+---------------+-------+---------+-----------------+---------------+------------+---------+---------+--------------+-------------------------+-------------------+----------+----------+-----+----------+-----------------+-------------------+-----------+--------+---------+---------+------+-----------+-----------+----------------+------------+------------+----------+--------+------------+----------+-------------+------------+-----------+------------+-------+---------+---------------+------------+----------+-----------------+-------------+------------+--------------+--------+-----------------+-------------------+--------------------+------------------+-----------------+----------+-------+------------------+---------------------+--------+------+----------+----------------

In [10]:
s3_path = "s3a://emart-datafabric-dev/oracle-db/"

df.write.mode("overwrite").parquet(s3_path)

In [11]:
!hdfs dfs -Dhadoop.security.credential.provider.path=jceks:///datafabric/credentials/aws.jceks -ls {s3_path}

2022-04-28 22:36:12,057 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-04-28 22:36:12,148 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-04-28 22:36:12,148 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
Found 2 items
-rw-rw-rw-   1 datafabric datafabric          0 2022-04-28 22:36 s3a://emart-datafabric-dev/oracle-db/_SUCCESS
-rw-rw-rw-   1 datafabric datafabric      94731 2022-04-28 22:36 s3a://emart-datafabric-dev/oracle-db/part-00000-282ce83c-e2c3-444c-a642-41a585d9e39a-c000.snappy.parquet
2022-04-28 22:36:14,319 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2022-04-28 22:36:14,319 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2022-04-28 22:36:14,319 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


In [12]:
spark.stop()